# <font color='blue'> Random forest classification model </font>

## Read input data, including ground truth data and images

In [ ]:
# Import GDAL, NumPy, and matplotlib
from osgeo import gdal, gdal_array
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Tell GDAL to throw Python exceptions, and register all drivers
gdal.UseExceptions()
gdal.AllRegister()

# Read in the image and ground data recalled as roi
img_ds = gdal.Open(r"path", gdal.GA_ReadOnly)
roi_ds = gdal.Open(r"path", gdal.GA_ReadOnly)

# gdal_array to read raster data as numeric array from file
img = np.zeros((img_ds.RasterYSize, img_ds.RasterXSize, img_ds.RasterCount),
              gdal_array.GDALTypeCodeToNumericTypeCode(img_ds.GetRasterBand(1).DataType)) 

for b in range (img.shape[2]):
    img[:, :, b] = img_ds.GetRasterBand(b+1).ReadAsArray()

roi = roi_ds.GetRasterBand(1).ReadAsArray().astype(np.int16)


# Display one band of image and ground truth (roi) data
plt.subplot(121)
plt.imshow(img[:, :, -2], cmap=plt.cm.Greys_r, vmin=-2000, vmax=10000) # vmin and vmax is defined based on your data
plt.title('Input Data')

plt.subplot(122)
plt.imshow(roi, cmap=plt.cm.Spectral)
plt.title("Ground Truth Data")

plt.show()

In [ ]:
if img_ds is not None: 
    print ("band count: " + str(img_ds.RasterCount))

## Using Skicit-learn to split ground truth and image data into training and testing sets


In [ ]:
from sklearn.model_selection import train_test_split

# Find how many valid entries are in the roi data -- i.e. how many ground truth data samples? in this data zero entries refer
# to non-valid enteries. 
n_samples = (roi > 0).sum()
print('There are {n} samples'.format(n=n_samples))

# What are the classification targets? i.e. classvalue 1 refers to irrigated areas and classvalue 627 refers to irrigated pixels 
targets = np.unique(roi[roi > 0])
print('The training data include {n} classes: {classes}'.format(n=targets.size, 
                                                                classes=targets))

# features : used to make predictions (e.g. the image inputs (img_ds))
# labels :  to be predicted (e.g. roi ground truth data)
features = img[roi > 0, :]  # include all bands
labels = roi[roi > 0]

# Split the data into training and testing sets
# train_test_split: Allowed inputs are lists, numpy arrays, scipy-sparse matrices or pandas dataframes.
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.3, random_state = 42)


# Make sure the split of data is correct
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

## Make sure the split of data is correct

In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

## Model tuning

#### Initialize a Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize our model
rf = RandomForestClassifier(random_state=42)

# select hyperparameters
n_estimators = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
max_depth = [5, 8, 15, 25, 30, None]
min_samples_split = [1, 2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10, 15]
max_features = [None, 'sqrt', 'log2']
bootstrap = [True, False]
oob_score = [True, False]

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf,
             max_features = max_features, bootstrap = bootstrap, oob_score = oob_score)

#### Finding best machine learning model hyperparameters

In [ ]:
# When having limitted resources: conduct a Randomized Search CV
from sklearn.model_selection import RandomizedSearchCV

randomF = RandomizedSearchCV(rf, hyperF, n_iter=10, n_jobs=-1, cv=3, verbose=1, random_state=42)

# Train Random Forest Model
bestF = randomF.fit(train_features, train_labels)
print("The mean accuracy of the model is:", bestF.score(test_features, test_labels))

In [ ]:
# get the best estimator that was chosen by the search
bestF.best_estimator_

# Parameter setting that gave the best results on the hold out data.
bestF.best_params_

# check the cv results
import pandas as pd
pd.DataFrame(bestF.cv_results_)

## Evaluate model performance

In [ ]:
def evaluate(model, test_features, test_labels):
    # Determine Performance Metrics 
    predictions = model.predict(test_features)
    
    # Get the Confusion Matrix, Classification report and ACcuracy
    from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

    print('Confusion Matrix:', confusion_matrix(test_labels, predictions))
    print (classification_report(test_labels, predictions))
    print("Accuracy:", accuracy_score(test_labels, predictions) * 100) 
    
    return predictions

evalu = evaluate(bestF, test_features, test_labels)

## Visualize confuse matrix 

In [ ]:
from sklearn.metrics import confusion_matrix


labels = {'class1': 'name1', 'class2': 'name2', 'classn':'namen'}
label_temp = [class1, class2, class3, classn] # put value of classes here


labels_list = [labels[key] for key in labels]

cm = confusion_matrix(test_labels, evalu, label_temp)
print(cm)

fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion Matrix \n', fontweight='bold', fontsize = 16)
fig.colorbar(cax)

ax.set_xticklabels([''] + labels_list, fontsize = 12)
ax.set_yticklabels([''] + labels_list, fontsize = 12)

for (i, j), z in np.ndenumerate(cm):
    ax.text(j, i, '{:0.0f}'.format(z), ha='center', va='center', color='white', fontweight='bold', fontsize = 12)

plt.xlabel('\n \n True', fontweight='bold', fontsize = 12)
plt.ylabel('Predicted', fontweight='bold', fontsize = 12)

plt.show()

## Get an idea of which spectral bands are important

In [ ]:
bands = list(range(9))

for b, imp in zip(bands, bestF.best_estimator_.feature_importances_):
    print('Band {b} importance: {imp}'.format(b = b, imp = imp))

# Predicting the rest of the image

In [ ]:
# Take our full image, and reshape into long 2d array (nrow * ncol, nband) for classification
new_shape = (img.shape[0] * img.shape[1], img.shape[2])

img_as_array = img[:, :, :].reshape(new_shape)
print('Reshaped from {o} to {n}'.format(o=img.shape,
                                        n=img_as_array.shape))

# Now predict for each pixel
class_prediction = bestF.predict(img_as_array)

# Reshape our classification map
class_prediction = class_prediction.reshape(img[:, :, 0].shape)

## Save the prediction as a raster file

In [ ]:
def save_raster (path, band_count, bands, srs, geoTransform, format='GTiff', dtype = gdal.GDT_Float32):
    rows,cols = bands.shape
    # Initialize driver & create file
    driver = gdal.GetDriverByName(format)
    dataset_out = driver.Create(path, cols, rows, 1, gdal.GDT_Float32)
    dataset_out.SetGeoTransform(geoTransform)
    dataset_out.SetProjection(srs)
    # Write file to disk
    dataset_out.GetRasterBand(1).WriteArray(bands)
    dataset_out = None
    
# Get the geo-reference information
geoTransform = img_ds.GetGeoTransform()
srs = img_ds.GetProjection()
bands = class_prediction
path = r"path" # for instance you can save output in .tif format

save_raster(path, 1, bands, srs, geoTransform)